In [67]:
import torch
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy.stats import normaltest
import matplotlib.pyplot as plt
from varname import nameof
from collections import Counter
from my_codes.weighted_ensembles.WeightedEnsemble import logit, pairwise_accuracies
from my_codes.weighted_ensembles.test_cifar10 import create_pairwise
from my_codes.weighted_ensembles.SimplePWCombine import bc
from my_codes.weighted_ensembles.predictions_evaluation import *

In [68]:
dn = torch.tensor(np.load("D:/skola/1/weighted_ensembles/tests_IM2012/comb_train/densenet201.npy"))
rn = torch.tensor(np.load("D:/skola/1/weighted_ensembles/tests_IM2012/comb_train/resnet152.npy"))
nn = torch.tensor(np.load("D:/skola/1/weighted_ensembles/tests_IM2012/comb_train/nasnetamobile.npy"))
tar = torch.tensor(np.load("D:/skola/1/weighted_ensembles/tests_IM2012/comb_train/targets.npy"))

In [69]:
class_from = 200
class_to = 210
classes = list(range(class_from,class_to))
dn = dn[(tar >= class_from) & (tar < class_to)][:, classes]
rn = rn[(tar >= class_from) & (tar < class_to)][:, classes]
nn = nn[(tar >= class_from) & (tar < class_to)][:, classes]
tar = tar[(tar >= class_from) & (tar < class_to)]
new_tar = torch.zeros_like(tar)
for cli, cl in enumerate(classes):
    new_tar[tar == cl] = cli

tar = new_tar

In [70]:
eps = 10e-5
MP = torch.cat([nn_p.unsqueeze(0) for nn_p in [dn, rn, nn]], dim=0)
c, n, k = MP.size()
coefs = torch.zeros(k, k, c + 1)
ldas = [[None for j in range(k)] for i in range(k)]
p_probs = torch.Tensor(n, k, k)
num_non_one = torch.sum(torch.abs(torch.sum(MP, dim=2) - 1.0) > eps).item()
if num_non_one > 0:
    MP = torch.nn.Softmax(dim=2)(MP)

In [71]:
pi = 0
for fc in range(k):
    for sc in range(fc + 1, k):
        SS = MP[:, (tar == fc) + (tar == sc)][:, :, [fc, sc]]
        PWP = torch.true_divide(SS[:, :, 0], torch.sum(SS, 2) + (SS[:, :, 0] == 0))
        LI = logit(PWP, eps)
        X = LI.transpose(0, 1)
        y = tar[(tar == fc) + (tar == sc)]
        mask_fc = (y == fc)
        mask_sc = (y == sc)
        y[mask_fc] = 1
        y[mask_sc] = 0
        
        clf = LinearDiscriminantAnalysis(solver='lsqr')
        clf.fit(X, y)
        coefs[fc, sc, :] = torch.cat((torch.tensor(clf.coef_).squeeze(),
                                            torch.tensor(clf.intercept_)))
        ldas[fc][sc] = clf

        pwacc = pairwise_accuracies(SS, y)
        print("Training pairwise accuracies for classes: " + str(fc) + ", " + str(sc) +
              "\n\tpairwise accuracies: " + str(pwacc) +
              "\n\tchosen coefficients: " + str(clf.coef_) +
              "\n\tintercept: " + str(clf.intercept_))
       
        print("\tcombined accuracy: " + str(clf.score(X, y)))
        
        pi += 1

Training pairwise accuracies for classes: 0, 1
	pairwise accuracies: tensor([0.9802, 1.0000, 0.9901])
	chosen coefficients: [[0.06063974 4.13531731 0.4835191 ]]
	intercept: [2.38680477]
	combined accuracy: 1.0
Training pairwise accuracies for classes: 0, 2
	pairwise accuracies: tensor([0.9901, 0.9901, 0.9604])
	chosen coefficients: [[ 0.8252583   1.31181557 -0.1698949 ]]
	intercept: [0.66068562]
	combined accuracy: 0.9900990099009901
Training pairwise accuracies for classes: 0, 3
	pairwise accuracies: tensor([0.9510, 0.9804, 0.9804])
	chosen coefficients: [[ 0.17101836  3.48351223 -0.235302  ]]
	intercept: [0.62295862]
	combined accuracy: 0.9803921568627451
Training pairwise accuracies for classes: 0, 4
	pairwise accuracies: tensor([0.9307, 0.9208, 0.8713])
	chosen coefficients: [[0.25748634 0.94268435 0.07909557]]
	intercept: [1.27683616]
	combined accuracy: 0.9504950495049505
Training pairwise accuracies for classes: 0, 5
	pairwise accuracies: tensor([1., 1., 1.])
	chosen coefficient

In [72]:
for fc in range(k):
    for sc in range(fc + 1, k):
        SS = MP[:, :, [fc, sc]]
        PWP = torch.true_divide(SS[:, :, 0], torch.sum(SS, 2) + (SS[:, :, 0] == 0))
        LI = logit(PWP, eps)
        X = LI.transpose(0, 1)
        PP = ldas[fc][sc].predict_proba(X)
        p_probs[:, sc, fc] = torch.from_numpy(PP[:, 0])
        p_probs[:, fc, sc] = torch.from_numpy(PP[:, 1])

In [73]:
comb = bc(p_probs.cuda()).cpu()
dn_pw = create_pairwise(dn)
rn_pw = create_pairwise(rn)
nn_pw = create_pairwise(nn)


In [74]:
dn_acc = compute_acc_topk(tar, dn, 1)
rn_acc = compute_acc_topk(tar, rn, 1)
nn_acc = compute_acc_topk(tar, nn, 1)
comb_acc = compute_acc_topk(tar, comb, 1)

print('\n'.join([nameof(pred) + " " + str(pred) for pred in [dn_acc, rn_acc, nn_acc, comb_acc]]))

pred 0.9223107569721115
pred 0.9243027888446215
pred 0.8904382470119522
pred 0.9342629482071713
